In [1]:
pip install opencv-python-headless numpy pandas moviepy


Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import pandas as pd
from moviepy.editor import VideoFileClip


# Define your color ranges here

In [8]:
def get_color_ranges():
    
    color_ranges = {
        'red': ([0, 120, 70], [10, 255, 255]),
        'blue': ([90, 120, 70], [120, 255, 255]),
        'yellow': ([20, 100, 100], [30, 255, 255]),
        'white': ([0, 0, 200], [180, 25, 255])
    }
    return color_ranges


# Function to determine the quadrant based on center coordinates

In [9]:
def get_quadrant(center_x, center_y, width, height):
    
    
    quadrant_x = center_x / width
    quadrant_y = center_y / height
    
    if quadrant_x < 0.5 and quadrant_y < 0.5:
        return 1
    elif quadrant_x >= 0.5 and quadrant_y < 0.5:
        return 2
    elif quadrant_x < 0.5 and quadrant_y >= 0.5:
        return 3
    else:
        return 4


In [56]:
def track_balls(frame, color_ranges, width, height, logs, prev_positions, entry_timestamps, exit_timestamps, timestamp):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    for color, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv_frame, np.array(lower), np.array(upper))
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            (x, y, w, h) = cv2.boundingRect(contour)
            if w * h > 100:  # Filter small contours
                center_x, center_y = x + w // 2, y + h // 2
                quadrant = get_quadrant(center_x, center_y, width, height)
                
                if color in prev_positions:
                    prev_quadrant = prev_positions[color]
                    if prev_quadrant != quadrant:
                        # Add exit log with timestamp if not already added
                        if color not in exit_timestamps or exit_timestamps[color] is None:
                            exit_timestamps[color] = timestamp
                            logs.append((exit_timestamps[color], prev_quadrant, color, 'Exit'))
                        
                        # Add entry log with timestamp if not already added
                        if color not in entry_timestamps or entry_timestamps[color] is None:
                            entry_timestamps[color] = timestamp
                            logs.append((entry_timestamps[color], quadrant, color, 'Entry'))
                
                prev_positions[color] = quadrant
                
                # Draw circle around the ball
                cv2.circle(frame, (center_x, center_y), w // 2, (0, 255, 255), 2)
                
                # Display color label above the ball
                text = f'{color.capitalize()} Ball'
                text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
                text_x = max(0, center_x - text_size[0] // 2)
                text_y = max(0, y - 10)
                cv2.putText(frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                
                # Display entry and exit timestamps
                if exit_timestamps[color] is not None:
                    exit_text = f'Exit Q{prev_quadrant} {exit_timestamps[color]:.2f}s'
                    cv2.putText(frame, exit_text, (text_x, text_y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                
                if entry_timestamps[color] is not None:
                    entry_text = f'Entry Q{quadrant} {entry_timestamps[color]:.2f}s'
                    cv2.putText(frame, entry_text, (text_x, text_y - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                
                # Reset timestamps after displaying them
                entry_timestamps[color] = None
                exit_timestamps[color] = None
    
    return frame


In [57]:
def process_frame(frame, color_ranges, width, height, logs, prev_positions, entry_timestamps, exit_timestamps, timestamp):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_processed = track_balls(frame_rgb, color_ranges, width, height, logs, prev_positions, entry_timestamps, exit_timestamps, timestamp)
    
    # Draw overlay texts for entry and exit events
    for color in color_ranges:
        if color in entry_timestamps and entry_timestamps[color] is not None:
            cv2.putText(frame_processed, f'{color.capitalize()} Ball Entry Q{prev_positions[color]} {entry_timestamps[color]:.2f}s',
                        (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            entry_timestamps[color] = None  # Reset entry timestamp to prevent repeated display
        if color in exit_timestamps and exit_timestamps[color] is not None:
            cv2.putText(frame_processed, f'{color.capitalize()} Ball Exit Q{prev_positions[color]} {exit_timestamps[color]:.2f}s',
                        (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            exit_timestamps[color] = None  # Reset exit timestamp to prevent repeated display
    
    return cv2.cvtColor(frame_processed, cv2.COLOR_RGB2BGR)

def process_video(input_video_path, output_video_path, output_log_path):
    clip = VideoFileClip(input_video_path)
    width, height = clip.size
    fps = clip.fps
    duration = clip.duration
    
    color_ranges = get_color_ranges()
    logs = []
    prev_positions = {color: None for color in color_ranges}
    entry_timestamps = {color: None for color in color_ranges}
    exit_timestamps = {color: None for color in color_ranges}
    
    def process_frame_wrapper(frame, timestamp):
        return process_frame(frame, color_ranges, width, height, logs, prev_positions, entry_timestamps, exit_timestamps, timestamp)
    
    processed_clip = clip.fl(lambda gf, t: process_frame_wrapper(gf(t), t))
    processed_clip.write_videofile(output_video_path, codec='libx264', audio=False)
    
    df_logs = pd.DataFrame(logs, columns=['Timestamp', 'Quadrant Number', 'Ball Colour', 'Type'])
    df_logs.to_csv(output_log_path, index=False)


In [58]:
def process_frame_wrapper(frame, timestamp):
    return process_frame(frame, color_ranges, width, height, logs, prev_positions, entry_timestamps, exit_timestamps, timestamp)


In [59]:
input_video_path = 'C:/Users/Ash/Downloads/input_video.mp4.mp4'
output_video_path = 'C:/Users/Ash/Downloads/output_video.mp4'
output_log_path = 'C:/Users/Ash/Downloads/output_log.csv'

process_video(input_video_path, output_video_path, output_log_path)


Moviepy - Building video C:/Users/Ash/Downloads/output_video.mp4.
Moviepy - Writing video C:/Users/Ash/Downloads/output_video.mp4



Moviepy - Done !
Moviepy - video ready C:/Users/Ash/Downloads/output_video.mp4
